# CMIP heat transport


# Setup

In [1]:
import os
import shutil
import glob
import pandas as pd
import numpy as np
import xarray as xr
import pickle as pkl

import cartopy
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.path as mpath

import cmocean.cm as cmo

import warnings
warnings.filterwarnings("ignore", message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")

In [2]:
## some magic to automatically reload my functions before running a new cell
%load_ext autoreload
## %reload_ext autoreload
%autoreload 1
%aimport my_functions

import my_functions as mf

## Figure settings

In [3]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 160

# %config InlineBackend.figure_formats = ['pdf']
%config InlineBackend.figure_formats = ['png']

## Load dictionaries

In [4]:
# Get the last 40 years of the 1% CO2 runs and the corresponding time period in piControl
with open('../pkl_files/period_l40_slice_in_pi.pkl', 'rb') as file:
    period_l40_slice_in_pi = pkl.load(file)
with open('../pkl_files/period_l40_start_year_in_pi.pkl', 'rb') as file:
    period_l40_start_year_in_pi = pkl.load(file)
with open('../pkl_files/period_l40_end_year_in_pi.pkl', 'rb') as file:
    period_l40_end_year_in_pi = pkl.load(file)
with open('../pkl_files/start_year_in_pi.pkl', 'rb') as file:
    start_year_in_pi = pkl.load(file)
    
with open('../pkl_files/period_l40_slice_in_co2.pkl', 'rb') as file:
    period_l40_slice_in_co2 = pkl.load(file)
with open('../pkl_files/period_l40_start_year_in_co2.pkl', 'rb') as file:
    period_l40_start_year_in_co2 = pkl.load(file)
with open('../pkl_files/period_l40_end_year_in_co2.pkl', 'rb') as file:
    period_l40_end_year_in_co2 = pkl.load(file) 
with open('../pkl_files/start_year_in_co2.pkl', 'rb') as file:
    start_year_in_co2 = pkl.load(file)


    
# Get each model's variant_id
with open('../pkl_files/variant_id.pkl', 'rb') as file:
    variant_id = pkl.load(file)
    
# Get each model's table_id
with open('../pkl_files/table_id.pkl', 'rb') as file:
    table_id = pkl.load(file)
    
# Get each model's grid_label
with open('../pkl_files/grid_label.pkl', 'rb') as file:
    grid_label = pkl.load(file)
    
# Colors corresponding to each model
with open('../pkl_files/mcolors.pkl', 'rb') as file:
    mcolors = pkl.load(file)

In [5]:
mcolors = {'ACCESS-ESM1-5': '#2f4f4f',
           'BCC-CSM2-MR': '#8b4513',
           'CanESM5': '#6b8e23',
           'CESM2': '#4b0082',
           'CMCC-ESM2': '#ff0000',
           'CNRM-ESM2-1': '#ffff00',
           'EC-Earth3-CC': 'black',
           'GFDL-ESM4': '#40e0d0',
           'GISS-E2-1-G': '#00ff00',
           'IPSL-CM6A-LR': '#0000ff',
           'MIROC-ES2L': '#ff00ff',
           'MPI-ESM1-2-LR': '#6495ed',
           'NorESM2-LM': '#ff1493',
           'UKESM1-0-LL': '#ffc0cb'}

## Create constants

In [15]:
# Cases / experiments
cases = ['piControl', '1pctCO2', '1pctCO2-rad', '1pctCO2-bgc']
cases_rad = ['1pctCO2', '1pctCO2-rad']

# Models with data in /tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/ and CMIPmerge/
# --> removed MRI-ESM2-0 because 1pctCO2 and 1pctCO2-rad Amon variables are the same
# --> removed CanESM5 because server could not be reached and model output not available (yet?)

# --> models with HT, VEG for 1pctCO2 and 1pctCO2-rad (i.e., the max number of models used)
models = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CMCC-ESM2',
          'CNRM-ESM2-1', 'EC-Earth3-CC', 'GFDL-ESM4', 'GISS-E2-1-G',
          'IPSL-CM6A-LR', 'MIROC-ES2L', 'MPI-ESM1-2-LR', 'NorESM2-LM',
          'UKESM1-0-LL']
#  --> models with HT, VEG, SW for all cases
models_all = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CMCC-ESM2',
              'CNRM-ESM2-1', 'GISS-E2-1-G', 'IPSL-CM6A-LR', 'MIROC-ES2L',
              'MPI-ESM1-2-LR', 'UKESM1-0-LL']
# --> only models with HT, VEG, SW for 1pctCO2 and 1pctCO2-rad
models_radsw = ['EC-Earth3-CC']
# --> only models with HT, VEG for 1pctCO2 and 1pctCO2-rad
models_rad = ['CESM2', 'GFDL-ESM4', 'NorESM2-LM']

# Variables
variables = ['evspsbl', 'hfls', 'hfss', 'hus', 'lai', 'pr', 'prsn', 'ps',
             'rlds',  'rlus', 'rlut', 'rsds', 'rsdt', 'rsus', 'rsut', 'ta',
             'tas', 'uas', 'vas']
variables_ht = ['hfls', 'hfss', 'hus', 'pr', 'prsn', 'ps', 'rlds', 'rlus',
                'rlut', 'rsds', 'rsdt', 'rsus', 'rsut', 'ta']
variables_vegsw = ['evspsbl', 'lai', 'tas', 'uas', 'vas']
variables_veg = ['evspsbl', 'lai', 'tas']
variables_2d = ['evspsbl', 'hfls', 'hfss', 'lai', 'pr', 'prsn', 'ps',
                'rlds',  'rlus', 'rlut', 'rsds', 'rsdt', 'rsus', 'rsut',
                'tas', 'uas', 'vas']
variables_3d = ['hus', 'ta']

# Months
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
months_abbr = ['J','F','M','A','M','J','J','A','S','O','N','D']

# Directory where CMIP6 model output lives on tiger
cmipdir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/'
cmipmergedir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPmerge/'
cmipgriddir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP1x1.25/'
cmipnhtdir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/'

constants = {
    'cases': cases,
    'cases_rad': cases_rad,
    'models': models,
    'models_all': models_all,
    'models_radsw': models_radsw,
    'models_rad': models_rad,
    'variables': variables,
    'variables_ht': variables_ht,
    'variables_vegsw': variables_vegsw,
    'variables_veg': variables_veg,
    'variables_2d': variables_2d,
    'variables_3d': variables_3d,
    'months': months,
    'months_abbr': months_abbr,
    'cmipdir': cmipdir,
    'cmipmergedir': cmipmergedir,
    'cmipgriddir': cmipgriddir,
    'cmipnhtdir': cmipnhtdir
}

# with open('../pkl_files/constants.pkl', 'wb') as file:
#     pkl.dump(constants, file, pkl.HIGHEST_PROTOCOL)

# Regrid

## New file periods

In [52]:
# Save new file periods as string for use in file naming and attributions --> 1pctCO2
with open('../regrid/newtimesco2.txt','w') as fout:
    for m in models:
        fout.write(period_l40_start_year_in_co2[m].replace('-','')+'-'+period_l40_end_year_in_co2[m].replace('-','')+'\n')
fout.close()

In [23]:
# Save new file periods as string for use in file naming and attributions --> piControl
with open('../regrid/newtimespi.txt','w') as fout:
    for m in models:
        try:
            fout.write(period_l40_start_year_in_pi[m].replace('-','')+'-'+period_l40_end_year_in_pi[m].replace('-','')+'\n')
        except:
            fout.write('\n')
fout.close()

## 2d variables, FULL + RAD

In [29]:
s = ''
s += ', '.join(models)+'\n'
s += ', '.join(variables_2d)+'\n'
s += ', '.join(cases_rad)+'\n'

for m in models:
    for v in variables_2d:
        for c in cases_rad:
            if not ( (m == 'CESM2' or m == 'GFDL-ESM4' or m == 'NorESM2-LM') and (v == 'uas' or v == 'vas') ):
                # print(m,' ',v,' ',c)
                f = v+'_'+table_id[v]+'_'+m+'_'+c+'_'+variant_id[m]+'_'+grid_label[m]+'.nc'
                s += cmipmergedir+m+'/'+f+'\n'

with open('../regrid/regridpaths_2d_fullrad.txt','w') as fout:
    fout.write(s)
fout.close()

## 3d variables, FULL + RAD

In [30]:
s = ''
s += ', '.join(models)+'\n'
s += ', '.join(variables_3d)+'\n'
s += ', '.join(cases_rad)+'\n'

for m in models:
    for v in variables_3d:
        for c in cases_rad:
            # print(m,' ',v,' ',c)
            f = v+'_'+table_id[v]+'_'+m+'_'+c+'_'+variant_id[m]+'_'+grid_label[m]+'.nc'
            s += cmipmergedir+m+'/'+f+'\n'

with open('../regrid/regridpaths_3d_fullrad.txt','w') as fout:
    fout.write(s)
fout.close()

# Generate arrays for the NCL script

In [7]:
s = '(/'
for m in models:
    s += '"'+m+'",'
s += '/)'
s

'(/"ACCESS-ESM1-5","BCC-CSM2-MR","CESM2","CMCC-ESM2","CNRM-ESM2-1","EC-Earth3-CC","GFDL-ESM4","GISS-E2-1-G","IPSL-CM6A-LR","MIROC-ES2L","MPI-ESM1-2-LR","NorESM2-LM","UKESM1-0-LL",/)'

In [8]:
s = '(/'

for m in models:
    if m != 'MRI-ESM2-0':
        s += str( 12 * (int(period_l40_start_year_in_co2[m][:4]) - int(start_year_in_co2[m][:4])) )
        s += ','
s += '/)'

s

'(/1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,/)'

In [11]:
s = '(/'
for m in models:
    s += '"'+grid_label[m]+'",' 
s += '/)'

s

'(/"gn","gn","gn","gn","gr","gr","gr1","gn","gr","gn","gn","gn","gn",/)'

In [12]:
s = '(/'
for m in models:
    s += '"'+variant_id[m]+'",' 
s += '/)'

s

'(/"r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f2","r1i1p1f1","r1i1p1f1","r101i1p1f1","r1i1p1f1","r1i1p1f2","r1i1p1f1","r1i1p1f1","r1i1p1f2",/)'

In [13]:
s = '(/'
for m in models:
    s += '"'+period_l40_start_year_in_co2[m].replace('-','')+'-'+period_l40_end_year_in_co2[m].replace('-','')+'",'
s += '/)'

s

'(/"020101-024012","195001-198912","010101-014012","195001-198912","195001-198912","195001-198912","010101-014012","195001-198912","195001-198912","195001-198912","195001-198912","010101-014012","195001-198912",/)'

# Compute HT

In [38]:
m = 'ACCESS-ESM1-5'
c = '1pctCO2'

# ht_dict = mf.get_heat_transports_seasonal(m, c, write=False)

In [37]:
for m in models:
    print(m)
    for c in cases_rad:
        print('  ',c)
        mf.get_heat_transports_seasonal(m, c, write=True)

ACCESS-ESM1-5
   1pctCO2


/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py

### (1/5) done with mht ###


/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py

### (2/5) done with oht ###


...

### (3/5) done with aht ###


/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


### (4/5) done with ahtmoist ###
### (5/5) done with ahtdry ###
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/ACCESS-ESM1-5/mht_Amon_ACCESS-ESM1-5_1pctCO2-rad_r1i1p1f1_zonal_020101-024012.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/ACCESS-ESM1-5/aht_Amon_ACCESS-ESM1-5_1pctCO2-rad_r1i1p1f1_zonal_020101-024012.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/ACCESS-ESM1-5/oht_Amon_ACCESS-ESM1-5_1pctCO2-rad_r1i1p1f1_zonal_020101-024012.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/ACCESS-ESM1-5/ahtmoist_Amon_ACCESS-ESM1-5_1pctCO2-rad_r1i1p1f1_zonal_020101-024012.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/ACCESS-ESM1-5/ahtdry_Amon_ACCESS-ESM1-5_1pctCO2-rad_r1i1p1f1_zonal_020101-024012.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/ACCESS-ESM1-5/storage_Amon_ACCESS-ESM1-5_1pctCO2-rad_r1i1p1f1_zonal_020101-024012.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/ACCESS-ESM1-5/storagemoist_Amon_ACCESS-ESM1-5_1pctCO2-rad_r1i1p1f1_zonal_020101-024012.nc
BCC-CSM2-MR
   1pctCO2


/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


### (1/5) done with mht ###
### (2/5) done with oht ###
### (3/5) done with aht ###
### (4/5) done with ahtmoist ###
### (5/5) done with ahtdry ###
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/BCC-CSM2-MR/mht_Amon_BCC-CSM2-MR_1pctCO2_r1i1p1f1_zonal_195001-198912.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/BCC-CSM2-MR/aht_Amon_BCC-CSM2-MR_1pctCO2_r1i1p1f1_zonal_195001-198912.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/BCC-CSM2-MR/oht_Amon_BCC-CSM2-MR_1pctCO2_r1i1p1f1_zonal_195001-198912.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/BCC-CSM2-MR/ahtmoist_Amon_BCC-CSM2-MR_1pctCO2_r1i1p1f1_zonal_195001-198912.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/BCC-CSM2-MR/ahtdry_Amon_BCC-CSM2-MR_1pctCO2_r1i1p1f1_zonal_195001-198912.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/BCC-CSM2-MR/storage_Amon_BCC-CSM2-MR_1pctCO2_r1i1p1f1_zonal_195001-198912.nc
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPnht/BCC-CSM2-MR/storagemoist_Amon_BCC-CSM2-MR_1pctCO2_r1i1p1f1_zonal_195001-198912.nc
   1pctCO2-rad
### (1/5) done wit